In [1288]:
#!/usr/bin/env python
# coding: utf-8

In [1289]:
import pandas as pd 
# pd.set_option("display.max_columns",30)
# pd.set_option("display.max_rows",3000)
import matplotlib.pyplot as plt 
import cufflinks as cf 
# cf.go_offline()
import numpy as np

### imputacion variables continuas
from sklearn.impute import SimpleImputer
from scipy.stats import ks_2samp

In [1290]:
df = pd.read_csv(r"C:\Users\jorge\OneDrive\Documentos\Documentos\UNAM\Minería de Datos\P3\DMUNAM2025\Practica_3\Dataset\titanic.csv")

In [1291]:
# df.shape

In [1292]:
# df.survived.value_counts(True)


# Podemos obervar gracias a nuestra funcion que tenemos 61% de personas que no sobrevivieron vs 38% que han sobrevivido

In [1293]:
# df

In [1294]:
def completitud_datos_nulos(df):
    return df.isnull().sum().sort_values(ascending=False) / df.shape[0]

In [1295]:
# completitud_datos_nulos(df)


    # Podemos observar que  body, cabin, boat y home.dest tien un porcentaje alto de nuelos, por lo cual podemos eliminarlas dado que no nos aportaran valor a nuestro modelo y si quisieramos llenarlo seria con valores NO CONFIABLES.

In [1296]:
columna_e = ['body', 'cabin', 'boat', 'home.dest']

df = df.drop(columna_e, axis=1)

In [1297]:
# completitud_datos_nulos(df)


    # podemos observar que ahora solo tenemos age y embarked con nulos, considerendo su porcentaje podemos imputar datos al modelo

In [1298]:
df = df[(~df["survived"].isnull()) & ~df["pclass"].isnull() & ~df["fare"].isnull() ]

In [1299]:
# df.shape

In [1300]:
# Podemos ver que nuestro registros han bajado a 1308 a consderacion de los 1310 que teniamos al inicio, asi mismo disminuyeron las columnas a 10 de las 14 inciales.

In [1301]:
# df


    # Podemos observar que ahora nuestro modelo se encuentra mucho mas limpio y con pocos nulos, por lo que vamos a porceder a imputar nuestras variables y asi llenar nuestros huecos.

In [1302]:
def complete_continuous_variables(df,col,strategy='median'):
    X = df[col].copy()
    im = SimpleImputer(strategy=strategy)
    Xi = pd.DataFrame(im.fit_transform(X),columns=col)
    l_ks = []
    for v in col:
        l_ks.append([v, ks_2samp(X[v].dropna(), Xi[v]).statistic])
    ks = pd.DataFrame(l_ks,columns=['feat','ks'])
    print(ks)
    print((ks.ks>=0.1).sum())
    #     df[col] = im.transform(df[col].copy())
    return df

In [1303]:
# completitud_datos_nulos(df)


    # Recordemos que tenemos variables nulas en embarked y age

In [1304]:
X = df[['age' ]]. copy()
im = SimpleImputer(strategy='median')
    # df['age'] = im.fit_transform(df[['age']].copy())

In [1305]:
# La razón por la cual usaremos SimpleImputer con median es porque a mediana es resistente a valores atípicos (outliers), por eso se prefiere cuando los datos pueden estar sesgados.

In [1306]:
# df['embarked'].value_counts(dropna=False)

In [1307]:
# Esto cuenta la cantidad de veces que aparece cada valor en la columna embarked, incluyendo los valores nulos (NaN).

In [1308]:
X_embarked = df[['embarked']].copy() 
im = SimpleImputer(strategy='most_frequent') 
    # df['embarked'] = im.fit_transform(X_embarked).ravel()


    # Se rellena con el dato mas frecuente.

In [1309]:
# df.shape

In [1310]:
# completitud_datos_nulos(df)

In [1311]:
# completitud_datos_nulos(df)


    # Al realizar todo este analsis ya podemos decir que no hay nulos en nuestras columnas

In [1312]:
# df.sample(n=30)


    # Tomamos una muestra aleatoria de nuestro DataFrame, dado que name es el nombre de la persona sobreviviente o fallecida, podemos eliminarlo dado que no aporta algo a nuestro modelo, asi como el ticket, ya que es una combinacion de numeros y letras.

In [1313]:
columna_ee = ['name', 'ticket']

df = df.drop(columna_ee, axis=1)

In [1314]:
# df.shape
    # completitud_datos_nulos(df)

In [1315]:
# df.sample(n=30)

In [1316]:
sexo = {'male':0, 'female' :1}
    # df['sex'] = df['sex'].map(sexo)

embarked = {'C':0, 'Q':1, 'S': 2}
    # df['embarked' ] = df['embarked' ].map(embarked)


    # La columna sex fue convertida a valores numéricos, asignando 0 para 'male' y 1 para 'female'.
    # 
    # La columna embarked, que representa el puerto de embarque, fue codificada como 0 para 'C' (Cherbourg), 1 para 'Q' (Queenstown) y 2 para 'S' (Southampton).

In [1317]:
# df.sample(n=30)

In [1318]:
varc = list(df.describe())

In [1319]:
varc = [x for x in varc if x not in 'survived']

In [1320]:
vard = [x for x in df.columns if x not in varc+['survived']]

In [1321]:
tgt = 'survived'

In [1322]:
# df[tgt].value_counts()


    # # Modelacion clasificacion

In [1323]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [1324]:
X = df[varc].copy() #TAD "Tabla Analitica de Datos"

In [1325]:
y = df[tgt].copy() #variable objetivo

In [1326]:
sc = MinMaxScaler()
Xs = pd.DataFrame(sc.fit_transform(X), columns=varc)

In [1327]:
y = y.astype(int)

In [1328]:
Xt, Xv, yt, yv = train_test_split(Xs,y, train_size=0.7)

In [1329]:
def entrenar(param, modelo, X,y):
    grid = RandomizedSearchCV(param_distributions=param,
    n_jobs=-1,
    n_iter=10,
    cv=4,
    estimator=modelo,
    error_score='raise')
    #     grid.fit(X,y)
    return grid, grid.best_estimator_, grid.best_score_, grid.best_params_

In [1330]:
# # Red Neuronal

In [1331]:
param_mlpc = dict(
    hidden_layer_sizes=[(a, b, c) for a in range(len(varc), len(varc)*2)
                                  for b in range(len(varc), len(varc)*2)
                                  for c in range(len(varc), len(varc)*2)],
    activation=['relu', 'tanh'],
    solver=['adam'],
    alpha=[0.0001, 0.001, 0.01]
)

In [1332]:
modelo_mlpc = MLPClassifier(solver='adam', max_iter=1000, random_state=42)
    # modelo_mlpc, best_estimator_mlpc, score_mlpc, params_mlpc = entrenar(param_mlpc, modelo_mlpc, Xt, yt)
    # metricas(Xt,Xv,yt,yv,modelo_mlpc)

In [1333]:
import pickle

In [1334]:
filename = './Najera_redneural.pkl'
    # with open(filename, 'wb') as file:
    #     pickle.dump(modelo_mlpc,file)
print(f'Modelo guardado en {filename}')

Modelo guardado en ./Najera_redneural.pkl


In [1335]:
# pwd

In [1336]:
# params_mlpc

In [1337]:
# score_mlpc

In [1338]:
# best_estimator_mlpc

In [1339]:
# modelo_mlpc
            # # Random Forest

In [1340]:
param = dict(n_estimators = range(2,10),
    max_depeth = range(2,6),
    max_features = range(2,len(varc)),
    criterion = ['gini', 'entropy'])

In [1341]:
param = dict(n_estimators=list(range(1, 100, 25)),
    criterion=['gini', 'entropy'],
    max_depth=[x for x in list(range(2, 5))] + [None],
    min_samples_split=[x for x in list(range(2, 4))],
    min_samples_leaf=[x for x in list(range(2, 4))],
    max_features=[None] + [i * .05 for i in list(range(2, 4))],
    max_leaf_nodes=list(range(2, 10)) + [None],
    min_impurity_decrease=[x * .10 for x in list(range(2, 4))],
    oob_score=[True,False],
    warm_start=[True, False],
    class_weight=[None, 'balanced'],
    max_samples=[None],)

In [1342]:
modelo = RandomForestClassifier()
    # modelo, best_estimator, score, params = entrenar(param, modelo, Xt, yt)
    # metricas(Xt,Xv,yt,yv,modelo)

In [1343]:
filename = './Najera_randomforest.pkl'
    # with open(filename, 'wb') as file:
    #     pickle.dump(modelo_mlpc, file)
print(f'Modelo guardado en {filename}')
            # # Ada Boost

Modelo guardado en ./Najera_randomforest.pkl


In [1344]:
from sklearn.ensemble import AdaBoostClassifier

In [1345]:
param_adab = dict(n_estimators = range(2,10),
    learning_rate = np.arange(0.1,1,0.1),
    algorithm = ['SAMME'])

In [1346]:
modelo_adab = AdaBoostClassifier()
    # modelo_adab, best_estimator, score, params = entrenar(param_adab, modelo_adab, Xt, yt)
    # metricas(Xt,Xv,yt,yv,modelo_adab)

In [1347]:
filename = './Najera_adaboost.pkl'
    # with open(filename, 'wb') as file:
    #     pickle.dump(modelo_mlpc, file)
print(f'Modelo guardado en {filename}')
            # # Analisis Discriminante

Modelo guardado en ./Najera_adaboost.pkl


In [1348]:
param_lda = dict(solver = ['svd', 'lsqr', 'eigen'])

In [1349]:
modelo = LinearDiscriminantAnalysis()
    # modelo, best_estimator, score, params = entrenar(param_lda, modelo, Xt, yt)
    # metricas(Xt,Xv,yt,yv,modelo)

In [1350]:
filename = './Najera_analisisdiscriminante.pkl'
    # with open(filename, 'wb') as file:
    #     pickle.dump(modelo_mlpc, file)
print(f'Modelo guardado en {filename}')
            # # Maquina Vector Soporte

Modelo guardado en ./Najera_analisisdiscriminante.pkl


In [1351]:
param_svc = dict(C = np.arange(0,2,0.1),
    kernel = ['linear','poly','rbf','sigmoid'],
    degree = range(2,6),
    gamma = ['scale','auto'], 
    probability = [True])

In [1352]:
# modelo_svc, best_estimator, score, params = entrenar(param_svc, modelo_svc, Xt, yt)
    # metricas(Xt,Xv,yt,yv,modelo_svc)

In [1353]:
filename = './Najera_maquinavectorsoporte.pkl'
    # with open(filename, 'wb') as file:
    #     pickle.dump(modelo_mlpc, file)
print(f'Modelo guardado en {filename}')
            # Conclusiones de los modelos
    # 
    # Red Nuronal: 85.8 % 
    # Ramdom Forest: 77.7%
    # Ada Bost:83.4%
    # Analisis de discriminate: 85.6%
    # Maquina vector soporte:86.6 %
def metricas(Xt, Xv, yt, yv, modelo):
    from sklearn.metrics import classification_report, confusion_matrix
    yhat_train = modelo.predict(Xt)
    yhat_val = modelo.predict(Xv)
    print('Entrenamiento')
    print(confusion_matrix(yt, yhat_train))
    print(classification_report(yt, yhat_train))
    print('Validación')
    print(confusion_matrix(yv, yhat_val))
    print(classification_report(yv, yhat_val))

Modelo guardado en ./Najera_maquinavectorsoporte.pkl


In [1354]:
print(Xt.shape)
print(yt.shape)
print(Xt.isnull().sum().sum(), 'valores nulos en Xt')
print(yt.isnull().sum(), 'valores nulos en yt')

(915, 5)
(915,)
188 valores nulos en Xt
0 valores nulos en yt


In [1355]:
# Rellenar valores nulos en Xt con la media de cada columna
Xt = Xt.fillna(Xt.mean())
Xv = Xv.fillna(Xv.mean())

In [1356]:
# Entrenar el modelo
modelo_mlpc.fit(Xt, yt)

# Guardar el modelo
import pickle

filename = './Najera_maquinavectorsoporte.pkl'

with open(filename, 'wb') as file:
    pickle.dump(modelo_mlpc, file)

print(f'Modelo guardado en {filename}')

# Mostrar métricas de entrenamiento y validación
metricas(Xt, Xv, yt, yv, modelo_mlpc)



Modelo guardado en ./Najera_maquinavectorsoporte.pkl
Entrenamiento
[[489  72]
 [167 187]]
              precision    recall  f1-score   support

           0       0.75      0.87      0.80       561
           1       0.72      0.53      0.61       354

    accuracy                           0.74       915
   macro avg       0.73      0.70      0.71       915
weighted avg       0.74      0.74      0.73       915

Validación
[[213  34]
 [ 79  67]]
              precision    recall  f1-score   support

           0       0.73      0.86      0.79       247
           1       0.66      0.46      0.54       146

    accuracy                           0.71       393
   macro avg       0.70      0.66      0.67       393
weighted avg       0.70      0.71      0.70       393



In [1357]:
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
import pickle

# Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
Xt_res, yt_res = smote.fit_resample(Xt, yt)

# Entrenar el modelo MLP con más iteraciones y mejor configuración
modelo_mlpc = MLPClassifier(
    max_iter=1500,
    random_state=42,
    activation='tanh',
    solver='adam'
)

modelo_mlpc.fit(Xt_res, yt_res)

# Guardar el modelo entrenado
filename = './Najera_maquinavectorsoporte.pkl'
with open(filename, 'wb') as file:
    pickle.dump(modelo_mlpc, file)

print(f'Modelo guardado en {filename}')

# Mostrar métricas usando el conjunto original de validación
metricas(Xt, Xv, yt, yv, modelo_mlpc)



Modelo guardado en ./Najera_maquinavectorsoporte.pkl
Entrenamiento
[[405 156]
 [152 202]]
              precision    recall  f1-score   support

           0       0.73      0.72      0.72       561
           1       0.56      0.57      0.57       354

    accuracy                           0.66       915
   macro avg       0.65      0.65      0.65       915
weighted avg       0.66      0.66      0.66       915

Validación
[[178  69]
 [ 56  90]]
              precision    recall  f1-score   support

           0       0.76      0.72      0.74       247
           1       0.57      0.62      0.59       146

    accuracy                           0.68       393
   macro avg       0.66      0.67      0.67       393
weighted avg       0.69      0.68      0.68       393



In [1358]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix

# Diccionario de modelos
modelos = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42),
    'LDA': LinearDiscriminantAnalysis()
}

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos.items():
    print(f'\n===== {nombre} =====')
    modelo.fit(Xt_res, yt_res)
    y_pred = modelo.predict(Xv)
    print('Matriz de confusión:')
    print(confusion_matrix(yv, y_pred))
    print('Reporte de clasificación:')
    print(classification_report(yv, y_pred))



===== Random Forest =====
Matriz de confusión:
[[188  59]
 [ 71  75]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.73      0.76      0.74       247
           1       0.56      0.51      0.54       146

    accuracy                           0.67       393
   macro avg       0.64      0.64      0.64       393
weighted avg       0.66      0.67      0.67       393


===== SVM =====
Matriz de confusión:
[[202  45]
 [ 64  82]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.76      0.82      0.79       247
           1       0.65      0.56      0.60       146

    accuracy                           0.72       393
   macro avg       0.70      0.69      0.69       393
weighted avg       0.72      0.72      0.72       393


===== AdaBoost =====
Matriz de confusión:
[[178  69]
 [ 59  87]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0   